<a href="https://colab.research.google.com/gist/TBeatrix/43ca56bac4956bb960a39df1cdc3d5d1/invase_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INVASE algorithm implementation in Pytorch**

Based on

Reference: Jinsung Yoon, James Jordon, Mihaela van der Schaar, 
           "IINVASE: Instance-wise Variable Selection using Neural Networks," 

           International Conference on Learning Representations (ICLR), 2019.

Paper Link: https://openreview.net/forum?id=BJg_roAcK7


---


In [ ]:
# Necessary packages
import numpy as np 
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# **Data generation**


---



In [ ]:
"""
Generating 6 synthetic datasets
x ~ N(0,I) where d = 11 or 100
y = 1/(1+logit) where logit for each synthetic dataset is
- syn1: logit = exp(x1 * x2)
- syn2: logit = exp(x3^2 + x4^2 + x5^2 + x6^2 -4)
- syn3: logit = -10 sin(2 * x7) + 2|x8| + x9 + exp(-x10) - 2.4
- syn4: If x11 < 0, follows syn1, else if x11 >= 0, follows syn2
- syn5: If x11 < 0, follows syn1, else if x11 >= 0, follows syn3
- syn6: If x11 < 0, follows syn2, else if x11 >= 0, follows syn3
"""

def generate_x (n, dim):
  """Generate the features (x).
  
  Args:
    - n: the number of samples
    - dim: the number of features (feature dimensions)
    
  Returns:
    - x: (n x dim) data points sample from N(0, 1)
  """
  x = np.random.randn(n, dim)
  return x

def generate_y (x, data_type):
  """Generate corresponding label (y) given feature (x).
  
  Args:
    - x: features
    - data_type: synthetic data type (syn1 to syn6)

  Returns:
    - y: corresponding labels
  """
  # number of samples
  n = x.shape[0]
    
  # Logit computation
  if data_type == 'syn1':
    logit = np.exp(x[:, 0]*x[:, 1])
  elif data_type == 'syn2':       
    logit = np.exp(np.sum(x[:, 2:6]**2, axis = 1) - 4.0) 
  elif data_type == 'syn3':
    logit = np.exp(-10 * np.sin(0.2*x[:, 6]) + abs(x[:, 7]) + \
                   x[:, 8] + np.exp(-x[:, 9])  - 2.4)     
  elif data_type == 'syn4':
    logit1 = np.exp(x[:, 0]*x[:, 1])
    logit2 = np.exp(np.sum(x[:, 2:6]**2, axis = 1) - 4.0) 
  elif data_type == 'syn5':
    logit1 = np.exp(x[:, 0]*x[:, 1])
    logit2 = np.exp(-10 * np.sin(0.2*x[:, 6]) + abs(x[:, 7]) + \
                    x[:, 8] + np.exp(-x[:, 9]) - 2.4) 
  elif data_type == 'syn6':
    logit1 = np.exp(np.sum(x[:,2:6]**2, axis = 1) - 4.0) 
    logit2 = np.exp(-10 * np.sin(0.2*x[:, 6]) + abs(x[:, 7]) + \
                    x[:, 8] + np.exp(-x[:, 9]) - 2.4) 
    
  # For syn4, syn5 and syn6 only
  if data_type in ['syn4', 'syn5', 'syn6']:
    # Based on X[:,10], combine two logits        
    idx1 = (x[:, 10]< 0)*1
    idx2 = (x[:, 10]>=0)*1    
    logit = logit1 * idx1 + logit2 * idx2    
        
  # Compute P(Y=0|X)
  prob_0 = np.reshape((logit / (1+logit)), [n, 1])
    
  # Sampling process
  y = np.zeros([n, 2])
  y[:, 0] = np.reshape(np.random.binomial(1, prob_0), [n,])
  y[:, 1] = 1-y[:, 0]

  return y


def generate_ground_truth(x, data_type):
  """Generate ground truth feature importance corresponding to the data type
     and feature.
  
  Args:
    - x: features
    - data_type: synthetic data type (syn1 to syn6)

  Returns:
    - ground_truth: corresponding ground truth feature importance
  """

  # Number of samples and features
  n, d = x.shape

  # Output initialization
  ground_truth = np.zeros([n, d])
        
  # For each data_type
  if data_type == 'syn1':
    ground_truth[:, :2] = 1
  elif data_type == 'syn2':
    ground_truth[:, 2:6] = 1
  elif data_type == 'syn3':
    ground_truth[:, 6:10] = 1
        
  # Index for syn4, syn5 and syn6
  if data_type in ['syn4', 'syn5', 'syn6']:        
    idx1 = np.where(x[:, 10]<0)[0]
    idx2 = np.where(x[:, 10]>=0)[0]
    ground_truth[:, 10] = 1
        
  if data_type == 'syn4':        
    ground_truth[idx1, :2] = 1
    ground_truth[idx2, 2:6] = 1
  elif data_type == 'syn5':        
    ground_truth[idx1, :2] = 1
    ground_truth[idx2, 6:10] = 1
  elif data_type == 'syn6':        
    ground_truth[idx1, 2:6] = 1
    ground_truth[idx2, 6:10] = 1
        
  return ground_truth
    
def generate_dataset(n = 10000, dim = 11, data_type = 'syn1', seed = 0):
  """Generate dataset (x, y, ground_truth).
  
  Args:
    - n: the number of samples
    - dim: the number of dimensions
    - data_type: synthetic data type (syn1 to syn6)
    - seed: random seed
    
  Returns:
    - x: features
    - y: labels
    - ground_truth: ground truth feature importance
  """

  # Seed
  np.random.seed(seed)

  # x generation
  x = generate_x(n, dim)
  # y generation
  y = generate_y(x, data_type)
  # ground truth generation
  ground_truth = generate_ground_truth(x, data_type)
  
  return x, y, ground_truth

# **Utility functions**
(1) Feature performance metrics

(2) Prediction performance metrics

(3) Bernoulli sampling


---



In [ ]:
def feature_performance_metric (ground_truth, importance_score):
  """Performance metrics for feature importance (TPR and FDR).
  Args:
    - ground_truth: ground truth feature importance
    - importance_score: computed importance scores for each feature
    
  Returns:
    - mean_tpr: mean value of true positive rate
    - std_tpr: standard deviation of true positive rate
    - mean_fdr: mean value of false discovery rate
    - std_fdr: standard deviation of false discovery rate
  """

  n = importance_score.shape[0]
  
  tpr = np.zeros([n, ])
  fdr = np.zeros([n, ])

  # For each sample
  for i in range(n):    
    # tpr   
    tpr_nom = np.sum(importance_score[i, :] * ground_truth[i, :])
    tpr_den = np.sum(ground_truth[i, :])
    tpr[i] = 100 * float(tpr_nom)/float(tpr_den + 1e-8)
        
    # fdr
    fdr_nom = np.sum(importance_score[i, :] * (1-ground_truth[i, :]))
    fdr_den = np.sum(importance_score[i,:])
    fdr[i] = 100 * float(fdr_nom)/float(fdr_den+1e-8)
    
  mean_tpr = np.mean(tpr)
  std_tpr = np.std(tpr)
  mean_fdr = np.mean(fdr)
  std_fdr = np.std(fdr)  
  
  return mean_tpr, std_tpr, mean_fdr, std_fdr

In [ ]:
def prediction_performance_metric (y_test, y_hat):
  """Performance metrics for prediction (AUC, APR, Accuracy).
  
  Args:
    - y_test: testing set labels
    - y_hat: prediction on testing set
    
  Returns:
    - auc: area under roc curve
    - apr: average precision score
    - acc: accuracy
  """
  
  auc = roc_auc_score (y_test[:, 1], y_hat[:, 1])
  apr = average_precision_score (y_test[:, 1], y_hat[:, 1])
  acc = accuracy_score (y_test[:, 1], 1.*(y_hat[:, 1] > 0.5))
  
  return auc, apr, acc

In [ ]:
def bernoulli_sampling (prob):
  """ Sampling Bernoulli distribution by given probability.
  
  Args:
    - prob: P(Y = 1) in Bernoulli distribution.
    
  Returns:
    - samples: samples from Bernoulli distribution
  """  

  n, d = prob.shape
  samples = np.random.binomial(1, prob, (n, d))
        
  return samples

# **INVASE Model**

---



In [ ]:
import torch
from torch import nn
import torch.optim as optim

In [ ]:
from torch.nn.modules import activation
class Invase():

    def __init__(self, x_train, y_train, model_type, model_parameters):
        
        self.lamda = model_parameters['lamda']
        self.actor_h_dim = model_parameters['actor_h_dim']
        self.critic_h_dim = model_parameters['critic_h_dim']
        self.n_layer = model_parameters['n_layer']
        self.batch_size = model_parameters['batch_size']
        self.iteration = model_parameters['iteration']
        if model_parameters['activation'] == "relu":
          self.activation = nn.ReLU()
        self.learning_rate = model_parameters['learning_rate']
        self.dim = x_train.shape[1]     
        self.label_dim = y_train.shape[1]
        self.model_type = model_type

        # Build and compile critic
        self.critic = self.build_critic()
        self.critic.optimizer = torch.optim.Adam(self.critic.parameters(), self.learning_rate, weight_decay=1e-3)
        self.critic.loss = nn.CrossEntropyLoss()
       
        # Build and compile the actor
        self.actor = self.build_actor()
        self.actor.optimizer = torch.optim.Adam(self.actor.parameters(), self.learning_rate, weight_decay=1e-3) 
        self.actor.loss = self.actor_loss

        if self.model_type == 'invase':
            # Build and compile the baseline
            self.baseline = self.build_baseline()
            self.baseline.optimizer = torch.optim.Adam(self.baseline.parameters(), self.learning_rate, weight_decay=1e-3)
            self.baseline.loss = nn.CrossEntropyLoss()


    def actor_loss(self, y_true, y_pred):
        '''
        y_true contains all outputs (actor, critic, basline) + ground truth
        '''
        # Actor output
        actor_out = y_true[:, :self.dim]
        # Critic output
        critic_out = y_true[:, self.dim:(self.dim+self.label_dim)]

        if self.model_type == 'invase':
            # Baseline output
            baseline_out = y_true[:, (self.dim+self.label_dim):(self.dim+2*self.label_dim)]
            # Ground truth label
            y_ground_truth = y_true[:, (self.dim+2*self.label_dim):]        
        elif self.model_type == 'invase_minus':
            # Ground truth label
            y_ground_truth = y_true[:, (self.dim+self.label_dim):]         

        # Critic loss
        critic_loss = -torch.sum(y_ground_truth * torch.log(critic_out + 1e-8), dim=1)  
        
        if self.model_type == 'invase':        
            # Baseline loss
            baseline_loss = -torch.sum(y_ground_truth * torch.log(baseline_out + 1e-8), dim=1)  
            # Reward
            Reward = -(critic_loss - baseline_loss)

        elif self.model_type == 'invase_minus':
            Reward = -critic_loss

        # Policy gradient loss computation. 
          
        custom_actor_loss = Reward * torch.sum(actor_out * torch.log(y_pred + 1e-8) + \
                                (1-actor_out)* torch.log(1-y_pred + 1e-8), dim=1) - \
                                self.lamda * torch.mean(y_pred, dim=1)

        # Custom actor loss
        custom_actor_loss = torch.mean(-custom_actor_loss)

        return custom_actor_loss

    def build_actor(self):

        class Actor(nn.Module):
          def __init__(self, params):
            super(Actor, self).__init__()
            # Params
            self.activation = params.activation
            self.n_layer = params.n_layer
            self.dim = params.dim
            self.actor_h_dim = params.actor_h_dim

            # Layers
            self.linear_in = nn.Linear(self.dim, self.actor_h_dim)
            self.linear_hidden = nn.Linear(self.actor_h_dim, self.actor_h_dim)
            self.linear_out = nn.Linear(self.actor_h_dim, self.dim)

          def forward(self, feature):
             x = self.activation(self.linear_in(feature))
             for i in range(self.n_layer -2):
               x = self.activation(self.linear_hidden(x))
             selection_probability = torch.sigmoid(self.linear_out(x))
             return selection_probability

        actor_model = Actor(self)
        return actor_model

    def build_critic(self):

        class Critic(nn.Module):
          def __init__(self, params):
            super(Critic, self).__init__()
            # Params
            self.activation = params.activation
            self.n_layer = params.n_layer
            self.label_dim = params.label_dim
            self.dim1 = params.dim
            self.critic_h_dim = params.critic_h_dim

            # Layers
            self.linear_in = nn.Linear(self.dim1, self.critic_h_dim)
            self.linear_hidden = nn.Linear(self.critic_h_dim, self.critic_h_dim)
            self.linear_out = nn.Linear(self.critic_h_dim, self.label_dim)
            self.batch_normalization = nn.BatchNorm1d(self.critic_h_dim)

          def forward(self, feature, selection):
               # Element wise multiplication
             
               critic_model_input = (feature * selection).float()
             
               x = self.activation(self.linear_in(critic_model_input))
               x = self.batch_normalization(x)
               for i in range(self.n_layer -2):
                  x = self.activation(self.linear_hidden(x))
                  x = self.batch_normalization(x)
               y_hat = nn.Softmax(dim=1)(self.linear_out(x))
               return y_hat
        critic_model = Critic(self)
        return critic_model


    def build_baseline(self):

         class Baseline(nn.Module):
          def __init__(self, params):
            super(Baseline, self).__init__()
            # Params
            self.activation = params.activation
            self.n_layer = params.n_layer
            self.label_dim = params.label_dim
            self.dim = params.dim
            self.baseline_h_dim = params.critic_h_dim # same as the critic

            # Layers
            self.linear_in = nn.Linear(self.dim, self.baseline_h_dim)
            self.linear_hidden = nn.Linear(self.baseline_h_dim, self.baseline_h_dim)
            self.linear_out = nn.Linear(self.baseline_h_dim, self.label_dim)
            self.batch_normalization = nn.BatchNorm1d(self.baseline_h_dim)

          def forward(self, feature):
               x = self.activation(self.linear_in(feature))
               x = self.batch_normalization(x)
               for i in range(self.n_layer -2):
                  x = self.activation(self.linear_hidden(x))
                  x = self.batch_normalization(x)
               y_hat = nn.Softmax(dim=1)(self.linear_out(x))
               return y_hat
         baseline_model = Baseline(self)
         return baseline_model

 #------------------------------------------------------------------------------#   

    def train(self, x_train, y_train):
   
        for i in range(self.iteration):
         
              ## Train critic
              # Select a random batch of samples
              idx = np.random.randint(0, x_train.shape[0], self.batch_size)
              x_batch = x_train[idx,:]
              y_batch = y_train[idx,:]
        for j in range(0, x_train.shape[0], self.batch_size):
              # Generate a batch of selection probability
              self.actor.eval()
              with torch.no_grad():
                selection_probability = self.actor(x_batch)                        
              # Sampling the features based on the selection_probability
              selection = bernoulli_sampling(selection_probability)  
         #for j in range(0, x_train.shape[0], self.batch_size): TODO
              # Critic loss
              self.critic.train()
              self.critic.optimizer.zero_grad()
              #Forward pass
              self.critic.output = self.critic(x_batch, selection)
              self.critic.loss_value = self.critic.loss(self.critic.output, y_batch)                        
              # Backward pass
              self.critic.loss_value.backward()
              # Update weights
              self.critic.optimizer.step()


              # Baseline output
              if self.model_type == 'invase':   
                  self.baseline.train()
                  # Baseline loss
                  self.baseline.optimizer.zero_grad()
                  # Forward pass
                  self.baseline.output = self.baseline(x_batch)
                  self.baseline.loss_value = self.baseline.loss(self.baseline.output, y_batch)                        
                  # Backward pass
                  self.baseline.loss_value.backward()
                  # Update weights
                  self.baseline.optimizer.step()

              ## Train actor
              # Use multiple things as the y_true: 
              # - selection, critic_out, baseline_out, and ground truth (y_batch)
              if self.model_type == 'invase':
    
                y_batch_final = torch.cat((torch.from_numpy(selection), self.critic.output,
                                          self.baseline.output, y_batch), axis = 1)
              elif self.model_type == 'invase_minus':
                y_batch_final = torch.cat((torch.from_numpy(selection), self.critic.output,
                                          y_batch), axis = 1)
              # Train the actor
              self.actor.train()
              self.actor.optimizer.zero_grad()
              # Forward pass
              self.actor.output = self.actor(x_batch)

              self.actor.loss_value = self.actor.loss(y_batch_final.detach(), self.actor.output)                       
              # Backward pass
              self.actor.loss_value.backward()
              # Update weights
              self.actor.optimizer.step()
                                                          

              # Print the progress
              if self.model_type == 'invase':
                  dialog = 'Iterations: ' + str(i) + \
                          ', critic loss: ' + str(self.critic.loss_value) + \
                          ', baseline loss: ' + str(self.baseline.loss_value) + \
                          ', actor loss: ' + str(self.actor.loss_value)
              elif self.model_type == 'invase_minus':
                  dialog = 'Iterations: ' + str(i) + \
                          ', critic loss: ' + str(self.critic.loss_value) + \
                          ', actor loss: ' + str(self.actor.loss_value)
              if i % 1000 == 0:
                  print(dialog)

    def importance_score(self, x):
        self.actor.eval()
        with torch.no_grad():
          feature_importance = self.actor(x)    
        return np.asarray(feature_importance)

    def predict(self, x_test):
         
        # Generate a batch of selection probability
        self.actor.eval()
        with torch.no_grad():
          selection_probability = self.actor(x_test)            
        # Sampling the features based on the selection_probability
        selection = bernoulli_sampling(selection_probability)   
        # Prediction 
        self.critic.eval()
        with torch.no_grad():
           y_hat = self.critic(x_test, selection)          
        return np.asarray(y_hat)


# **Main**


---

(1) Data generation

(2) Train INVASE or INVASE-

(3) Evaluate INVASE on ground truth feature importance and prediction

In [ ]:
#Paraméterel megadása a modellnek
class args:
  data_type: str
  train_no: int
  test_no: int
  dim : int
  model_type : str
  actor_h_dim : int
  critic_h_dim :int
  n_layer: int
  batch_size : int
  iteration : int
  activation : str
  learning_rate : float
  lamda : float

# Inputs for the main function
args.data_type = 'syn1'
args.train_no = 10000
args.test_no = 10000
args.dim = 11
args.model_type = 'invase'
args.actor_h_dim = 100
args.critic_h_dim = 200
args.n_layer = 3
args.batch_size = 1000
args.iteration = 10000
args.activation = 'relu'
args.learning_rate = 0.0001
args.lamda = 0.1


In [ ]:
# Generate dataset
x_train, y_train, g_train = generate_dataset (n = args.train_no, 
                                              dim = args.dim, 
                                              data_type = args.data_type, 
                                              seed = 0)

x_test, y_test, g_test = generate_dataset (n = args.test_no,
                                            dim = args.dim, 
                                            data_type = args.data_type, 
                                            seed = 0)

model_parameters = {'lamda': args.lamda,
                    'actor_h_dim': args.actor_h_dim, 
                    'critic_h_dim': args.critic_h_dim,
                    'n_layer': args.n_layer,
                    'batch_size': args.batch_size,
                    'iteration': args.iteration, 
                    'activation': args.activation, 
                    'learning_rate': args.learning_rate}

# Train the model

x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
model = Invase(x_train_tensor, y_train_tensor, args.model_type, model_parameters)

model.train(x_train_tensor, y_train_tensor)    

Iterations: 0, critic loss: tensor(0.7140, grad_fn=<DivBackward1>), baseline loss: tensor(0.7145, grad_fn=<DivBackward1>), actor loss: tensor(0.0434, grad_fn=<MeanBackward0>)
Iterations: 0, critic loss: tensor(0.7068, grad_fn=<DivBackward1>), baseline loss: tensor(0.7026, grad_fn=<DivBackward1>), actor loss: tensor(-0.0505, grad_fn=<MeanBackward0>)
Iterations: 0, critic loss: tensor(0.7114, grad_fn=<DivBackward1>), baseline loss: tensor(0.7156, grad_fn=<DivBackward1>), actor loss: tensor(0.1297, grad_fn=<MeanBackward0>)
Iterations: 0, critic loss: tensor(0.7107, grad_fn=<DivBackward1>), baseline loss: tensor(0.7032, grad_fn=<DivBackward1>), actor loss: tensor(-0.0960, grad_fn=<MeanBackward0>)
Iterations: 0, critic loss: tensor(0.7093, grad_fn=<DivBackward1>), baseline loss: tensor(0.7164, grad_fn=<DivBackward1>), actor loss: tensor(0.1679, grad_fn=<MeanBackward0>)
Iterations: 0, critic loss: tensor(0.7062, grad_fn=<DivBackward1>), baseline loss: tensor(0.7006, grad_fn=<DivBackward1>), 

In [ ]:
## Evaluation
# Compute importance score
x_test_tensor = torch.from_numpy(x_test).float()

g_hat = model.importance_score(x_test_tensor)
importance_score = 1.*(g_hat > 0.5)

In [ ]:
# Evaluate the performance of feature importance
mean_tpr, std_tpr, mean_fdr, std_fdr = \
feature_performance_metric(g_test, importance_score)
  
# Print the performance of feature importance    
print('TPR mean: ' + str(np.round(mean_tpr,1)) + '\%, ' + \
      'TPR std: ' + str(np.round(std_tpr,1)) + '\%, ')
print('FDR mean: ' + str(np.round(mean_fdr,1)) + '\%, ' + \
      'FDR std: ' + str(np.round(std_fdr,1)) + '\%, ')

In [ ]:
# Predict labels
y_hat = model.predict(x_test_tensor)
  
# Evaluate the performance of feature importance
auc, apr, acc = prediction_performance_metric(y_test, y_hat)
  
# Print the performance of feature importance    
print('AUC: ' + str(np.round(auc, 3)) + \
      ', APR: ' + str(np.round(apr, 3)) + \
      ', ACC: ' + str(np.round(acc, 3)))

performance = {'mean_tpr': mean_tpr, 'std_tpr': std_tpr,
                'mean_fdr': mean_fdr, 'std_fdr': std_fdr,
                'auc': auc, 'apr': apr, 'acc': acc}
print(performance)